In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -mkdir /tmp/output

In [3]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/output/

In [4]:
%%hive

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 9.83 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.001 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.432 seconds


In [5]:
%%hive
DROP TABLE IF EXISTS resultado;

CREATE TABLE resultado 
AS
SELECT  ano,
        letra,
        COUNT(letra)
FROM (
        SELECT 
            YEAR(c4) AS ano,
            letra
        FROM
            tbl0
        LATERAL VIEW explode(c5) tbl0 AS letra
    ) t0

GROUP BY ano, letra;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM resultado;

DROP TABLE IF EXISTS resultado;
OK
Time taken: 0.009 seconds
CREATE TABLE resultado 
AS
SELECT  ano,
        letra,
        COUNT(letra)
FROM (
        SELECT 
            YEAR(c4) AS ano,
            letra
        FROM
            tbl0
        LATERAL VIEW explode(c5) tbl0 AS letra
    ) t0
GROUP BY ano, letra;
Query ID = root_20200118220723_e7b982a8-6a4b-4525-a6fc-2de19dd7e856
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579385130637_0001, Tracking URL = http://282e0d2d9a5d:8088/proxy/application_1579385130637_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579385130637_0001
Hadoop job information fo

In [6]:
!hdfs dfs -copyToLocal /tmp/output output;

In [7]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.553 seconds


In [8]:
%hive_quit